In [1]:
# so we can use packages from parent directory
import sys
sys.path.append("..")

In [5]:
import torch
import torch.nn as nn
import numpy as np
from monroe_data import MonroeData, MonroeDataEntry, Color # last two for reading pkl file

from models import CaptionEncoder

In [7]:
# for reloading
import monroe_data
import caption_featurizers
import color_featurizers
import models

In [6]:
import importlib

In [47]:
importlib.reload(caption_featurizers)
importlib.reload(color_featurizers)

<module 'color_featurizers' from '../color_featurizers.py'>

In [48]:
from caption_featurizers import CaptionFeaturizer
from color_featurizers import ColorFeaturizer, color_phi_fourier

In [113]:
importlib.reload(models)

<module 'models' from '../models.py'>

In [114]:
from models import CaptionEncoder, LiteralListener

In [8]:
# train data
train_data = MonroeData("../data/csv/train_corpus_monroe.csv", "../data/entries/train_entries_monroe.pkl")
dev_data = MonroeData("../data/csv/dev_corpus_monroe.csv", "../data/entries/dev_entries_monroe.pkl")

In [ ]:
class Featurizer:
    def to_features(self, data_entry, **kwargs):
        pass
    
    def construct_featurizer(self, all_data, **kwargs):
        pass

In [184]:
class FeatureHandler:
    """
    This class handles the interface between the data, the feature functions, and the model.
    Basically what it does is the following:
    
    1. Convert MonroeDataEntry to a list of np.array's per speaker using the caption and color feature
       functions along with any other feature functions that the model needs
    2. Converts the MonroeDataEntry to the prediction targets also by applying some user-specified function
       to each of the data entries
    3. Handles color order randomization
    
    It does this for both the train and assessment datasets. From here, you should be able
    to call a model's fit method with the resutls of a `train_features` as X and `train_targets` as y
    """
    def __init__(self, train_data, test_data, caption_phi, color_phi, extra_featurizers=[], 
                 target_fn=None, randomized_colors=True):
        """
        tain_data - training data (type: MonroeData)
        test_data - assessment data (type: MonroeData)
        caption_phi - caption feature function (type: CaptionFeaturizer). Should not have called `construct_featurizer`
                      method yet
        color_phi   - color feature function (type: ColorFeaturizer)
        extra_featurizers - list any other feature functions to include. Each should have a `to_features`
                            method that takes a MonroeDataEntry to a feature
        target_fn - function for mapping MonroeDataentry (and permuation if randomized_colors is true)
                    to a *single value*
        randomized_colors - True if color order should be randomized, False if should be fixed with target first.
        
        """
        self.caption_featurizer = caption_phi
        self.caption_featurizer.construct_featurizer(train_data)
        
        self.color_featurizer = color_phi
        
        self.extra_featurizers = extra_featurizers
        self.train_data = train_data
        self.test_data = test_data
        self.randomized_colors = randomized_colors
        
        if target_fn is None:
            # by default just use the color at the target index as the target
            if self.randomized_colors:
                self.target_fn = lambda de, color_perm: np.where(color_perm==de.target_idx)[0]
            else:
                self.target_fn = lambda de: de.target_idx # should be 0, but just in case
        
        # for keeping track of where colors ended up if randomized
        self.train_color_permutations = []
        self.test_color_permutations = []
        
        # only construct caption index once
        self.constructed_index = False
     
    
    def get_features(self, data, construct=False):
        features = []
        for data_entry in data:
            entry_features = []
            if self.caption_featurizer is not None:
                _, idx_features = self.caption_featurizer.to_string_features(data_entry.caption, construct) # construct index while training
                entry_features.append(idx_features)
            
            if self.color_featurizer is not None:
                color_features = self.color_featurizer.to_color_features(data_entry.colors)
                
                if self.randomized_colors:
                    color_features, permutations = self.color_featurizer.shuffle_colors(color_features)
                    if construct:
                        self.train_color_permutations.append(permutations)
                    else:
                        self.test_color_permutations.append(permutations)
                    
                entry_features.append(color_features)
                
            for featurizer in self.extra_featurizers:
                entry_features.append(featurizer.to_features(data_entry))
            features.append(entry_features)
                
        return features
    
    def train_features(self):
        """
        Wrapper function for get_features that calls specifically with train data. Should
        only be called once because it also constructs the index
        """
        features = self.get_features(self.train_data, construct=(not self.constructed_index)) # will only construct index the first time
        self.constructed_index = True
        return features
                                 
                
    def test_features(self):
        """
        Wrapper function for get_features that calls specifically with assess data
        """
        return self.get_features(self.test_data)
    
    def get_targets(self, data, permutations=[]):
        """
        Given data, iterates through it and extracts whatever the target of the model prediction
        will be by calling self.target_fn on the entry. If we are going to be predicting color
        indices we need to know where the target color index ended up. To this end, we also pass
        in the permuted indices list in permutations.
        
        A way around this would be to actually change the raw entry and get the target index with
        entry.target_idx, but that kind of scares me...
        """
        if len(permutations) == 0 and self.randomized_colors:
            print("Make sure to call feature function before target function so color permutations can be used when generating targets")
            return
            
        targets = []
        # we need to pass in the permutations to the target functions
        # so we know where each color ended up - this is kind of ugly
        # but needed if our task is predicting colors (not needed otherwise)
        # but included when color is randomized
        for i, data_entry in enumerate(data):
            if len(permutations) == 0:
                targets.append(self.target_fn(data_entry))
            else:
                targets.append(self.target_fn(data_entry, permutations[i]))
        return targets
    
    
    def train_targets(self):
        """
        Wrapper function for get_targets that calls specifically with train data
        """
        self.get_targets(self.train_data, self.train_color_permutations)
            
        
    def test_targets(self):
        """
        Wrapper function for get_targets that calls specifically with assess data
        """
        return self.get_targets(self.test_data, self.test_color_permutations)
    
    
        
        

In [104]:
caption_phi = CaptionFeaturizer()
color_phi = ColorFeaturizer(color_phi_fourier, "rgb")

In [105]:
feature_handler = FeatureHandler(train_data, dev_data, caption_phi, color_phi)

In [106]:
%%time
train_features_2 = feature_handler.train_features()

CPU times: user 6.57 s, sys: 33.5 ms, total: 6.6 s
Wall time: 6.64 s


In [70]:
train_features[2]

[array([ 0,  7,  8,  9,  1,  7, 10,  4,  5]),
 array([[ 1.0000000e+00, -8.0320752e-01,  2.9028466e-01,  8.0320752e-01,
         -1.0000000e+00,  8.0320752e-01,  2.9028466e-01, -8.0320752e-01,
          1.0000000e+00, -6.0551107e-01,  1.2271538e-02,  5.8579785e-01,
         -9.6043050e-01,  6.0551107e-01, -1.2271538e-02, -9.3733901e-01,
          9.6043050e-01, -6.0551107e-01, -2.6671275e-01,  7.8834641e-01,
         -9.9969882e-01,  3.5989505e-01,  2.6671275e-01, -7.8834641e-01,
          8.4485358e-01, -3.5989505e-01, -2.6671275e-01,  0.0000000e+00,
         -5.9569931e-01,  9.5694035e-01, -5.9569931e-01, -1.2246469e-16,
          5.9569931e-01, -9.5694035e-01,  5.9569931e-01,  0.0000000e+00,
         -7.9583693e-01,  9.9992472e-01, -8.1045717e-01, -2.7851969e-01,
          7.9583693e-01, -9.9992472e-01,  3.4841868e-01,  2.7851969e-01,
         -7.9583693e-01,  9.6377605e-01, -6.1523157e-01,  2.4541229e-02,
          9.3299282e-01, -9.6377605e-01,  6.1523157e-01,  5.3499764e-01,
     

In [107]:
train_targets_2 = feature_handler.train_targets()

In [66]:
train_targets[:10]

[array([0]),
 array([1]),
 array([2]),
 array([2]),
 array([0]),
 array([1]),
 array([1]),
 array([2]),
 array([1]),
 array([1])]

In [108]:
%%time
dev_features = feature_handler.test_features()

In [109]:
dev_targets = feature_handler.dev_targets()

In [115]:
model = LiteralListener(CaptionEncoder, num_epochs=10)
model.init_model(embed_dim = 100, hidden_dim=100, vocab_size=feature_handler.caption_featurizer.caption_indexer.size,
                 color_dim=54)

In [94]:
#model.fit(train_features, train_targets)

---EPOCH 0---
0m 0s (0:0 0.00%) 0.0000
0m 9s (0:1000 6.38%) 1.2166
0m 20s (0:2000 12.77%) 0.9582
0m 30s (0:3000 19.15%) 0.9452
0m 42s (0:4000 25.53%) 0.8948
0m 53s (0:5000 31.92%) 0.9285
1m 4s (0:6000 38.30%) 0.7167
1m 14s (0:7000 44.69%) 0.6538
1m 25s (0:8000 51.07%) 0.7567
1m 36s (0:9000 57.45%) 0.5829
1m 47s (0:10000 63.84%) 0.5665
1m 58s (0:11000 70.22%) 0.5253
2m 9s (0:12000 76.60%) 0.5837
2m 20s (0:13000 82.99%) 0.6169
2m 32s (0:14000 89.37%) 0.5434
2m 43s (0:15000 95.75%) 0.6253
---EPOCH 1---
2m 51s (1:0 0.00%) 0.0003
3m 2s (1:1000 6.38%) 0.4815
3m 13s (1:2000 12.77%) 0.5982
3m 25s (1:3000 19.15%) 0.6874
3m 37s (1:4000 25.53%) 0.6747
3m 48s (1:5000 31.92%) 0.7112
3m 59s (1:6000 38.30%) 0.5173
4m 11s (1:7000 44.69%) 0.4642
4m 22s (1:8000 51.07%) 0.6016
4m 34s (1:9000 57.45%) 0.4483
4m 45s (1:10000 63.84%) 0.4519
4m 57s (1:11000 70.22%) 0.4306
5m 8s (1:12000 76.60%) 0.5471
5m 20s (1:13000 82.99%) 0.5106
5m 32s (1:14000 89.37%) 0.4140
5m 44s (1:15000 95.75%) 0.5656
---EPOCH 2---
5m

In [112]:
#model.save_model("../model/literal_listener_10epoch.params")

In [116]:
model.load_model("../model/literal_listener_10epoch.params")

In [117]:
%%time
model_outputs = model.predict(dev_features)

In [118]:
model_outputs[:5]

[tensor([[-19.9660, -42.4203,   0.0000]]),
 tensor([[-5.0325e+01, -3.8782e+00, -2.0905e-02]]),
 tensor([[-20.6142,   0.0000, -23.8036]]),
 tensor([[-96.3790, -85.4181,   0.0000]]),
 tensor([[-69.7986, -38.0655,   0.0000]])]

In [119]:
dev_targets[:5]

[array([2]), array([2]), array([1]), array([2]), array([2])]

In [120]:
model_outputs[0].numpy()

array([[-19.96595 , -42.420288,   0.      ]], dtype=float32)

In [123]:
model_outputs_np = np.array([mo.view(-1).numpy() for mo in model_outputs])

In [127]:
model_outputs_np[:5]

array([[-1.9965950e+01, -4.2420288e+01,  0.0000000e+00],
       [-5.0324768e+01, -3.8782482e+00, -2.0904541e-02],
       [-2.0614235e+01,  0.0000000e+00, -2.3803600e+01],
       [-9.6378983e+01, -8.5418053e+01,  0.0000000e+00],
       [-6.9798553e+01, -3.8065472e+01,  0.0000000e+00]], dtype=float32)

In [129]:
np.argmax(model_outputs_np[:5], axis=1)

array([2, 2, 1, 2, 2])

In [137]:
targets_np = np.array(dev_targets[:5]).flatten()

In [133]:
sum(np.array(dev_targets[:5]).flatten() == np.argmax(model_outputs_np[:5], axis=1))

5

In [134]:
def accuracy(model_outputs, targets):
    model_outputs = np.argmax(model_outputs, axis=1)
    targets = np.array(targets).flatten()
    
    num_correct = sum(model_outputs == targets)
    return num_correct / len(targets)

In [136]:
accuracy(model_outputs_np, dev_targets)

0.7913848117421826

In [140]:
model_outputs_np[:5]

array([[-1.9965950e+01, -4.2420288e+01,  0.0000000e+00],
       [-5.0324768e+01, -3.8782482e+00, -2.0904541e-02],
       [-2.0614235e+01,  0.0000000e+00, -2.3803600e+01],
       [-9.6378983e+01, -8.5418053e+01,  0.0000000e+00],
       [-6.9798553e+01, -3.8065472e+01,  0.0000000e+00]], dtype=float32)

In [148]:
model_outputs_np[:5][np.arange(len(targets_np)), targets_np]

array([ 0.        , -0.02090454,  0.        ,  0.        ,  0.        ],
      dtype=float32)

In [143]:
targets_np.T

array([2, 2, 1, 2, 2])

In [165]:
argstuff = {'hi':2, 'lo':4}

In [170]:
def hi(i, **kwargs):
    print("The" ,i, "th caller")
    print(kwargs)

In [173]:
hi(3, **{})

The 3 th caller
{}


In [156]:
def to_score(model_outputs, dev_targets):
    return model_outputs[np.arange(len(model_outputs)), dev_targets]

In [157]:
model_scores = to_score(model_outputs_np, np.array(dev_targets).flatten())

In [158]:
model_scores

array([ 0.        , -0.02090454,  0.        , ...,  0.        ,
        0.        ,  0.        ], dtype=float32)

In [151]:
np.array(dev_targets).flatten()

array([2, 2, 1, ..., 2, 1, 2])

In [152]:
np.arange(len(model_outputs_np))

array([    0,     1,     2, ..., 15667, 15668, 15669])

In [155]:
model_outputs_np[np.arange(len(model_outputs_np)), np.array(dev_targets).flatten()]

array([ 0.        , -0.02090454,  0.        , ...,  0.        ,
        0.        ,  0.        ], dtype=float32)

In [189]:
model_scores

array([ 0.        , -0.02090454,  0.        , ...,  0.        ,
        0.        ,  0.        ], dtype=float32)

In [159]:
from evaluation import score_model

In [160]:
score_model(dev_data, model_scores)

(0.2405230644216966, 1.4965601575829739e-05)

In [190]:
score_model(dev_data, np.exp(model_scores))

(0.32298518345881677, 3.947468963173441e-09)

In [183]:
np.where(np.array([0, 2, 1]) == 0)[0]

array([0])

In [186]:
torch.tensor([1])

tensor([1])

In [192]:
np.empty([len(model_outputs), 3])

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       ...,
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])